In [1]:
import pandas as pd

In [3]:
dsum = pd.read_csv("https://raw.githubusercontent.com/centre-for-health-informatics/SPORNW-2025-EMR-Workshop/refs/heads/master/synthetic_discharge_summaries.csv")
print(dsum.shape)
dsum.head(2)

(20, 3)


,Unnamed: 0,title,summary
0,0,1. Acute Appendicitis,The patient is a 24-year-old male with no sign...
1,1,2. Pneumonia (Community-Acquired),The patient is a 67-year-old female with a his...
